In [2]:
import os
import zipfile
from docx import Document
import pandas as pd
from PyPDF2 import PdfReader
import sqlite3
from fpdf import FPDF

<div style="font-size: 20px">
Загрузка содержимого офисных документов в базу полнотекстового индекса.
Краулинг хранилища файлов doc, docx, xls, xlsx, pdf, а также их вложенностей в архивы (zip, rar, 7z)
с последующей загрузкой текстовой информации в FTI-базу.

In [4]:
#Папка для хранения файлов
out_dir = "new_files"
os.makedirs(out_dir, exist_ok=True)

#Генерация DOCX
doc_names = ["doc1.docx", "doc2.docx"]
for doc_name in doc_names:
    doc = Document()
    doc.add_heading('Текст' + doc_name, level=1)
    doc.add_paragraph("Содержимое " + doc_name)
    doc.save(os.path.join(out_dir, doc_name))

#Генерация XLSX
xlsx_names = ["sheet1.xlsx", "sheet2.xlsx"]
for xlsx_name in xlsx_names:
    df = pd.DataFrame({
        "Col1": ["Head1", "Head2", "Head3"],
        "Col2": ["Data1", "Data2", "Data3"]
    })
    df.to_excel(os.path.join(out_dir, xlsx_name), index=False)

#Генерация PDF
pdf_names = ["album1.pdf", "album2.pdf"]
for pdf_name in pdf_names:
      
    pdf = FPDF()
    pdf.add_page()
    pdf.add_font("Arial", '', "C:/Windows/Fonts/arial.ttf", uni=True)
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Содержимое " + pdf_name, ln=True)
    pdf.output(os.path.join(out_dir, pdf_name))

#Генерация ZIP архивов
with zipfile.ZipFile(os.path.join(out_dir, 'arch1.zip'), 'w') as zipf:
    for doc_name in doc_names:
        zipf.write(os.path.join(out_dir, doc_name), doc_name)

print("Сгенерировано")

Сгенерировано


In [6]:
def extract_text_from_docx(path):
    doc = Document(path)
    return '\n'.join(par.text for par in doc.paragraphs)

def extract_text_from_pdf(path):
    text = ""
    with open(path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def crawl_directory(dirs):
    data = []
    for root, dirs, files in os.walk(dirs):
        for file in files:
            path = os.path.join(root, file)
            if file.endswith(".docx"):
                text = extract_text_from_docx(path)
            elif file.endswith(".pdf"):
                text = extract_text_from_pdf(path)
            elif file.endswith(".xlsx"):
                df = pd.read_excel(path)
                text = df.to_string()
            else:
                continue
            
            data.append({"file_name": file, "file_path": path, "content": text})

    return pd.DataFrame(data)

dirs = "new_files"
df = crawl_directory(dirs)

#Сохранение в CSV
df.to_csv("craw_data.csv", index=False)
print("Cохранено в craw_data.csv")

Cохранено в craw_data.csv


In [13]:
#Подключаемся к базе данных (или создаем ее)
conn = sqlite3.connect('dock.db')
cur = conn.cursor()

#Создаем таблицу
cur.execute('''
    CREATE TABLE IF NOT EXISTS documents (
        id INTEGER PRIMARY KEY,
        file_name TEXT,
        file_path TEXT,
        content TEXT
    )
''')

#Загружаем данные из CSV
df = pd.read_csv("craw_data.csv")
df.to_sql('dock', conn, if_exists='append', index=False)

conn.commit()
print("Импортировано")
conn.close()

def search_documents(query):
    conn = sqlite3.connect('dock.db')
    cursor = conn.cursor()
    
    cursor.execute("SELECT file_name, content FROM dock WHERE content LIKE ?", ('%' + query + '%',))
    results = cursor.fetchall()
    
    conn.close()
    return results

#Пример поиска
searched_documents = search_documents("Содержимое")
print("Результаты:")
for doc in searched_documents:
    print(f"Документ: {doc[0]}, Информация: {doc[1][:100]}...")  #Показывает первые 100 символов

Импортировано
Результаты:
Документ: album1.pdf, Информация: Содержимое album1.pdf
...
Документ: album2.pdf, Информация: Содержимое album2.pdf
...
Документ: doc1.docx, Информация: Текстdoc1.docx
Содержимое doc1.docx...
Документ: doc2.docx, Информация: Текстdoc2.docx
Содержимое doc2.docx...
Документ: album1.pdf, Информация: Содержимое album1.pdf
...
Документ: album2.pdf, Информация: Содержимое album2.pdf
...
Документ: doc1.docx, Информация: Текстdoc1.docx
Содержимое doc1.docx...
Документ: doc2.docx, Информация: Текстdoc2.docx
Содержимое doc2.docx...
Документ: album1.pdf, Информация: Содержимое album1.pdf
...
Документ: album2.pdf, Информация: Содержимое album2.pdf
...
Документ: doc1.docx, Информация: Текстdoc1.docx
Содержимое doc1.docx...
Документ: doc2.docx, Информация: Текстdoc2.docx
Содержимое doc2.docx...
Документ: album1.pdf, Информация: Содержимое album1.pdf
...
Документ: album2.pdf, Информация: Содержимое album2.pdf
...
Документ: doc1.docx, Информация: Текстdoc1.docx
Содержимое doc